# Amazon Bedrock Knowledge Bases와 RAG Assessment(RAGAS) 프레임워크로 Q&A 애플리케이션 구축 및 평가

### 컨텍스트

이 노트북에서는 Amazon Bedrock Knowledge Bases가 제공하는 Retrieve API와 LangChain, RAGAS를 함께 사용해 Q&A 애플리케이션을 구축하고 응답을 평가하는 방법을 살펴봅니다. 지식 베이스에서 유사도 검색을 기반으로 원하는 수만큼 문서 청크를 가져오고, Anthropic Claude로 프롬프트를 구성한 뒤, faithfulness, answer_relevancy, context_recall, context_precision, context_entity_recall, answer_similarity, answer_correctness, harmfulness, maliciousness, coherence, correctness, conciseness 등 다양한 평가 지표로 응답을 검증합니다.

### Amazon Bedrock Knowledge Bases 소개

Knowledge Base를 사용하면 Amazon Bedrock의 Foundation Model(FM)을 기업 데이터에 안전하게 연결해 Retrieval Augmented Generation(RAG)을 구현할 수 있습니다. 추가 데이터에 접근하면 모델이 더 관련성 있고 컨텍스트에 특화된 정확한 응답을 생성하면서 FM을 지속적으로 재학습할 필요가 없습니다. Knowledge Base에서 검색된 모든 정보에는 출처가 제공되어 투명성을 높이고 환각을 줄입니다. 콘솔에서 Knowledge Base를 만드는 방법은 [문서](!https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html)를 참고하세요.

### 패턴

이 솔루션은 Retrieval Augmented Generation(RAG) 패턴으로 구현할 수 있습니다. RAG는 언어 모델 외부의 데이터(비모수 데이터)를 검색해 관련 데이터를 프롬프트에 추가합니다. 여기서는 이전 노트북 또는 콘솔에서 생성한 Knowledge Base로 RAG를 수행합니다. 

### 사전 준비

질문에 답하려면 문서를 처리해 Knowledge Base에 저장해야 합니다. 이 노트북에서는 Amazon Bedrock Knowledge Bases를 생성하기 위해 `synthetic dataset for 10K financial reports`를 사용합니다. 

1. 문서(데이터 소스)를 Amazon S3 버킷에 업로드합니다.
2. Amazon Bedrock Knowledge Bases는 [01_create_ingest_documents_test_kb_multi_ds.ipynb](/knowledge-bases/01-rag-concepts/01_create_ingest_documents_test_kb_multi_ds.ipynb)를 사용해 생성합니다.
3. Knowledge Base ID를 기록해 둡니다.

<!-- ![data_ingestion.png](./images/data_ingestion.png) -->
<img src="./images/data_ingestion.png" width=50% height=20% />

#### 노트북 워크스루

이 노트북에서는 Amazon Bedrock Knowledge Bases가 제공하는 `Retrieve API`를 사용합니다. 이 API는 사용자 질의를 임베딩으로 변환하고 Knowledge Base를 검색하며, 의미 기반 검색 결과 위에 사용자 정의 워크플로를 구축할 수 있도록 관련 결과를 반환합니다. `Retrieve API`의 출력에는 `검색된 텍스트 청크`, 소스 데이터의 `위치 유형`과 `URI`, 검색의 관련성 `점수`가 포함됩니다. 

생성된 텍스트 청크를 원래 프롬프트와 결합해 `anthropic.claude-3-haiku-20240307-v1:0` 모델에 전달합니다.

마지막으로 RAGAS를 사용해 생성된 응답을 평가합니다. 이때 faithfulness, answer relevancy, context precision 등 지표를 사용하며 평가에는 `anthropic.claude-3-sonnet-20240229-v1:0`을 활용합니다.

### 질문하기

<!-- ![retrieveapi.png](./images/retrieveAPI.png) -->
<img src="./images/retrieveAPI.png" width=50% height=20% />

#### 평가
1. RAGAS를 사용해 다음 지표로 평가합니다:
    1. **Faithfulness:** 주어진 컨텍스트 대비 생성된 답변의 사실 일관성을 측정합니다. 답변과 검색된 컨텍스트를 기반으로 계산하며 값은 0에서 1 사이이고 높을수록 좋습니다.
    2. **Answer Relevance:** 생성된 답변이 프롬프트와 얼마나 관련 있는지 평가합니다. 불완전하거나 불필요한 정보를 포함한 답변에는 낮은 점수가 부여되며, 질문·컨텍스트·답변을 사용해 계산합니다. 실무에서는 0~1 범위에 속하지만 코사인 유사도 특성상 수학적으로 보장되지는 않습니다.
    3. **Context Precision:** 컨텍스트에 포함된 그라운드 트루스 관련 항목이 상위에 위치하는지 평가합니다. 이상적으로는 관련 청크가 상위 순위에 나타나야 하며, 질문·그라운드 트루스·컨텍스트를 사용해 계산하며 값은 0~1 범위입니다.
    4. **Context Recall:** 검색된 컨텍스트가 주석 답변(그라운드 트루스)과 얼마나 일치하는지 측정합니다. 값은 0~1 범위이며 높을수록 좋습니다.
    5. **Context entities recall:** 컨텍스트에서 검색된 엔터티를 기반으로 컨텍스트의 재현율을 측정합니다.
    6. **Answer Similarity:** 생성된 답변과 그라운드 트루스 답변 간의 유사성을 평가합니다.
    7. **Answer Correctness:** 생성된 답변이 얼마나 정확한지 측정합니다.
    8. **Harmfulness / Maliciousness:** 생성된 응답이 유해하거나 악의적인지 평가합니다.
    9. **Coherence / Correctness / Conciseness:** 응답이 얼마나 일관되고 정확하며 간결한지 평가합니다.

`evaluate()` 함수를 사용해 모든 지표를 한 번에 계산하면 RAGAS가 병렬 비동기 방식으로 LLM을 여러 번 호출하므로 InvokeModel ThrottlingException이 발생할 수 있습니다. 이를 방지하기 위해 `single_turn_ascore()`를 사용합니다.

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

#### 위에서 설치한 패키지를 적용하려면 커널을 재시작하세요

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### 필요한 패키지 설정 절차

1. Amazon Bedrock Knowledge Bases가 제공하는 Retrieve API를 사용하기 위해 Foundation Model을 호출하는 `bedrock-runtime` 클라이언트와 `bedrock-agent-runtime` 클라이언트를 초기화하는 데 필요한 라이브러리를 임포트합니다. 
2. LangChain을 임포트해 다음 작업을 수행합니다: 
   1. RAG 패턴으로 질의 완성을 수행하기 위해 `anthropic.claude-3-haiku-20240307-v1:0` 모델을 LLM으로 초기화합니다. 
   2. RAG 평가를 수행하기 위해 `anthropic.claude-3-sonnet-20240229-v1:0` 모델을 LLM으로 초기화합니다. 
   3. Knowledge Base와 통합된 LangChain retriever를 초기화합니다. 
   4. 노트북 후반부에서는 `RetrieverQAChain`으로 LLM과 retriever를 감싸 Q&A 애플리케이션을 구성합니다.

In [ ]:
%store -r kb_id
# kb_id = "<<knowledge_base_id>>" # Replace with your knowledge base id here.anthropic.claude-3-haiku-20240307-v1:0

In [ ]:
import boto3
from botocore.client import Config
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})

bedrock_client = boto3.client('bedrock-runtime')
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config
                              )
region, account_id

In [ ]:
from langchain.llms.bedrock import Bedrock
from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain_aws.embeddings.bedrock import BedrockEmbeddings
from langchain_aws.retrievers.bedrock import AmazonKnowledgeBasesRetriever
from langchain.chains import RetrievalQA

llm_for_text_generation = ChatBedrock(model_id="anthropic.claude-3-haiku-20240307-v1:0", client=bedrock_client)

llm_for_evaluation = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", client=bedrock_client)

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0",client=bedrock_client)

### Retrieve API 처리 흐름 

LangChain에서 `AmazonKnowledgeBasesRetriever` 객체를 생성하면 Amazon Bedrock Knowledge Bases의 `Retrieve API`를 호출해 사용자 질의를 임베딩으로 변환하고 Knowledge Base를 검색한 뒤 관련 결과를 반환합니다. 이를 통해 의미 기반 검색 결과 위에 사용자 정의 워크플로를 세밀하게 구성할 수 있습니다. `Retrieve API` 출력에는 `retrieved text chunks`, 소스 데이터의 `location type`과 `URI`, 검색 관련성 `scores`가 포함됩니다. 

In [ ]:
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 5}},
        # endpoint_url=endpoint_url,
        # region_name="us-east-1",
        # credentials_profile_name="<profile_name>",
    )

`score`: 반환된 각 텍스트 청크의 점수를 확인하면 질의와의 유사도를 파악할 수 있습니다.

### RetrievalQA 체인을 사용한 모델 호출과 응답 생성 

모델을 호출하고 응답을 확인합니다.

Question = `Provide a list of few risks for Octank financial in numbered list without description."`

Ground truth answer = 
```
1. Commodity Prices
2. Foreign Exchange Rates 
3. Equity Prices
4. Credit Risk
5. Liquidity Risk
...
...
```

In [ ]:
query = "Provide a list of few risks for Octank financial in numbered list without description."

qa_chain = RetrievalQA.from_chain_type(
    llm=llm_for_text_generation, retriever=retriever, return_source_documents=True
)

response = qa_chain.invoke(query)
print(response["result"])

## 평가 데이터 준비

RAGAS는 참조(reference)가 없는 평가 프레임워크를 지향하기 때문에 평가 데이터 세트를 준비하는 데 필요한 절차가 최소화되어 있습니다. `question`과 `ground_truths` 쌍만 준비하면 아래 예시처럼 나머지 정보를 추론으로 생성할 수 있습니다. `context_recall` 지표에 관심이 없다면 `ground_truths`를 제공하지 않아도 되며, 이 경우 준비해야 할 것은 `questions`뿐입니다.

In [ ]:
import pandas as pd

questions = [
        "What was the primary reason for the increase in net cash provided by operating activities for Octank Financial in 2021?",
        "In which year did Octank Financial have the highest net cash used in investing activities, and what was the primary reason for this?",
        "What was the primary source of cash inflows from financing activities for Octank Financial in 2021?",
        "Calculate the year-over-year percentage change in cash and cash equivalents for Octank Financial from 2020 to 2021.",
        "Based on the information provided, what can you infer about Octank Financial's overall financial health and growth prospects?"
]
ground_truths = [
    "The increase in net cash provided by operating activities was primarily due to an increase in net income and favorable changes in operating assets and liabilities.",
    "Octank Financial had the highest net cash used in investing activities in 2021, at $360 million, compared to $290 million in 2020 and $240 million in 2019. The primary reason for this was an increase in purchases of property, plant, and equipment and marketable securities.",
    "The primary source of cash inflows from financing activities for Octank Financial in 2021 was an increase in proceeds from the issuance of common stock and long-term debt.",
    "To calculate the year-over-year percentage change in cash and cash equivalents from 2020 to 2021: \
    2020 cash and cash equivalents: $350 million \
    2021 cash and cash equivalents: $480 million \
    Percentage change = (2021 value - 2020 value) / 2020 value * 100 \
    = ($480 million - $350 million) / $350 million * 100 \
    = 37.14% increase",
    "Based on the information provided, Octank Financial appears to be in a healthy financial position and has good growth prospects. The company has consistently increased its net cash provided by operating activities, indicating strong profitability and efficient management of working capital. Additionally, Octank Financial has been investing in long-term assets, such as property, plant, and equipment, and marketable securities, which suggests plans for future growth and expansion. The company has also been able to finance its growth through the issuance of common stock and long-term debt, indicating confidence from investors and lenders. Overall, Octank Financial's steady increase in cash and cash equivalents over the past three years provides a strong foundation for future growth and investment opportunities."
]

answers = []
contexts = []

for query in questions:
  answers.append(qa_chain.invoke(query)["result"])
  contexts.append([docs.page_content for docs in retriever.invoke(query)])

# Create dataset df
dataset_df = pd.DataFrame(list(zip(questions, ground_truths, answers)),
                 columns=['questions', 'ground_truths', 'answers'])
# dataset_df

## RAG 애플리케이션 평가
먼저 `ragas.metrics`에서 사용할 지표를 모두 임포트합니다. 그런 다음 `evaluate()` 함수에 준비한 데이터 세트와 사용할 지표를 전달하면 됩니다.

하지만 모든 지표를 한 번에 `evaluate()` 함수로 계산하면 RAGAS 프레임워크가 병렬 비동기 방식으로 LLM을 다수 호출하므로 InvokeModel ThrottlingException이 발생할 수 있습니다. 이를 피하기 위한 여러 방법이 있으며, 여기서는 `single_turn_ascore()`를 사용합니다.

In [ ]:
# evaluation using ragas

from tqdm.notebook import tqdm
import pandas as pd

# ragas libraries
from ragas import SingleTurnSample 
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# ragas metrics
from ragas.metrics import (LLMContextPrecisionWithReference,
                            LLMContextRecall)
from ragas.metrics._factual_correctness import FactualCorrectness

metrics = { 'context_precision': LLMContextPrecisionWithReference,
             'context_recall': LLMContextRecall, 
             'factual_correctness': FactualCorrectness
            }

# Model Configuration
llm_for_evaluation = LangchainLLMWrapper(llm_for_evaluation)
bedrock_embeddings = LangchainEmbeddingsWrapper(bedrock_embeddings)

# Create an empty DataFrame
df_retrieve = pd.DataFrame(columns=list(metrics.keys()))

score_list = []
# Outer progress bar for samples
with tqdm(total=len(questions), desc="Evaluating samples") as pbar_samples:
    for idx, (question, ground_truth, context, answer) in enumerate(zip(questions, ground_truths, contexts, answers)):
        score_dict = {}
        # Inner progress bar for metrics
        with tqdm(total=len(metrics), desc=f"Sample {idx+1} metrics", leave=False) as pbar_metrics:
            for k, v in metrics.items():
                scorer = v(llm=llm_for_evaluation)
                sample = SingleTurnSample(
                            user_input=question,
                            response=answer,
                            reference=ground_truth,
                            retrieved_contexts=context, 
                            )
                score_result = await scorer.single_turn_ascore(sample)
                score_dict[k] = score_result
                pbar_metrics.update(1)
        score_list.append(score_dict)
        pbar_samples.update(1)

# convert score_list to df
df_metrics = pd.DataFrame(score_list)

# merge metric df with dataset df
results = pd.merge(dataset_df, df_metrics, left_index=True, right_index=True)

In [ ]:
results

> 참고: 위 점수는 RAG 애플리케이션 성능을 상대적으로 파악하는 용도로 참고해야 하며, 단독 지표로 사용하지 마세요. 또한 여기서는 평가를 위해 5개의 질문/답변 쌍만 사용했지만, 실제로는 문서의 다양한 측면을 다룰 수 있도록 충분한 데이터를 사용하는 것이 좋습니다.

점수를 바탕으로 RAG 워크플로의 다른 구성요소를 살펴보고 추가 최적화를 고려하세요. 예를 들어 청킹 전략 조정, 프롬프트 지침 개선, numberOfResults 값을 늘려 추가 컨텍스트를 제공하는 방법 등이 있습니다. 

<div class="alert alert-block alert-warning">
<b>참고:</b> 비용이 발생하지 않도록 KB, OSS 인덱스 및 관련 IAM 역할과 정책을 반드시 삭제하세요.
</div>